In [1]:
#1.1. Базовый класс Fuzzy с перегруженными операторами
import numpy as np

class Fuzzy:
    def __init__(self, val=0):
        val = np.array(val) if isinstance(val, (list, np.ndarray)) else np.array([val])
        self.val = val.astype('float32')
    
    def __invert__(self):          # ~x NOT
        return Fuzzy(1 - self.val)
    
    def __and__(self, other):      # x & y AND
        return Fuzzy(np.minimum(self.val, other.val))
    
    def __or__(self, other):       # x | y OR
        return Fuzzy(np.maximum(self.val, other.val))
    
    def __eq__(self, other):
        return Fuzzy(self.val == other.val)
    
    def __str__(self):
        return f"{self.val[0]}" if len(self.val) == 1 else f"{self.val}"
    
    def minmax(self):
        return (self.val.min(), self.val.max())
    
    def __gt__(self, other):
        return Fuzzy(np.minimum(np.ones_like(self.val), 1 - self.val + other.val))

In [2]:
#1.2. 5 примеров для системы управления сплит-системой

In [3]:
#Пример 1: Базовые логические операции для температуры
print("=== Пример 1: Базовые операции ===")
# Температурные условия
temp_low = Fuzzy([0, 0, 0, 0.5, 0.5, 0.5, 1, 1, 1])      # Низкая температура
temp_high = Fuzzy([1, 1, 1, 0.5, 0.5, 0.5, 0, 0, 0])     # Высокая температура

print("Низкая темп:", temp_low)
print("Высокая темп:", temp_high)
print("НЕ низкая:", ~temp_low)
print("Низкая И высокая:", temp_low & temp_high)
print("Низкая ИЛИ высокая:", temp_low | temp_high)

=== Пример 1: Базовые операции ===
Низкая темп: [0.  0.  0.  0.5 0.5 0.5 1.  1.  1. ]
Высокая темп: [1.  1.  1.  0.5 0.5 0.5 0.  0.  0. ]
НЕ низкая: [1.  1.  1.  0.5 0.5 0.5 0.  0.  0. ]
Низкая И высокая: [0.  0.  0.  0.5 0.5 0.5 0.  0.  0. ]
Низкая ИЛИ высокая: [1.  1.  1.  0.5 0.5 0.5 1.  1.  1. ]


In [4]:
#Пример 2: Комбинированные условия для управления

In [5]:
print("\n=== Пример 2: Комбинированные условия ===")
# Условия для включения мощного охлаждения
high_temp = Fuzzy([0, 0, 0.2, 0.5, 0.8, 1, 1, 1, 1])        # Высокая температура
high_humidity = Fuzzy([0, 0, 0, 0.3, 0.6, 0.9, 1, 1, 1])    # Высокая влажность

# Мощное охлаждение включается при высокой температуре И высокой влажности
power_cooling = high_temp & high_humidity
print("Высокая темп:", high_temp)
print("Высокая влаж:", high_humidity)
print("Мощное охлаждение (И):", power_cooling)


=== Пример 2: Комбинированные условия ===
Высокая темп: [0.  0.  0.2 0.5 0.8 1.  1.  1.  1. ]
Высокая влаж: [0.  0.  0.  0.3 0.6 0.9 1.  1.  1. ]
Мощное охлаждение (И): [0.  0.  0.  0.3 0.6 0.9 1.  1.  1. ]


In [6]:
#Пример 3: Альтернативные условия работы

In [7]:
print("\n=== Пример 3: Альтернативные условия ===")
# Условия для включения вентиляции
normal_temp = Fuzzy([0, 0.3, 0.7, 1, 0.7, 0.3, 0, 0, 0])    # Нормальная температура
normal_humidity = Fuzzy([0, 0, 0.4, 0.8, 1, 0.8, 0.4, 0, 0]) # Нормальная влажность

# Вентиляция включается при нормальной температуре ИЛИ нормальной влажности
ventilation = normal_temp | normal_humidity
print("Нормальная темп:", normal_temp)
print("Нормальная влаж:", normal_humidity)
print("Вентиляция (ИЛИ):", ventilation)


=== Пример 3: Альтернативные условия ===
Нормальная темп: [0.  0.3 0.7 1.  0.7 0.3 0.  0.  0. ]
Нормальная влаж: [0.  0.  0.4 0.8 1.  0.8 0.4 0.  0. ]
Вентиляция (ИЛИ): [0.  0.3 0.7 1.  1.  0.8 0.4 0.  0. ]


In [8]:
#Пример 4: Проверка тождеств для системы управления

In [9]:
print("\n=== Пример 4: Проверка тождеств ===")
vals = np.linspace(0, 1, 11, dtype='float32')

temp = Fuzzy(vals)
humidity = Fuzzy(vals[::-1])  # Обратный порядок для разнообразия

# Дистрибутивность: A & (B | C) == (A & B) | (A & C)
A, B, C = temp, humidity, Fuzzy([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.9, 0.8])

left_side = A & (B | C)
right_side = (A & B) | (A & C)
distributive_check = (left_side == right_side)

print("Дистрибутивность A & (B | C) == (A & B) | (A & C):")
print(f"Минимальное совпадение: {distributive_check.minmax()[0]:.1f}")


=== Пример 4: Проверка тождеств ===
Дистрибутивность A & (B | C) == (A & B) | (A & C):
Минимальное совпадение: 1.0


In [10]:
#Пример 5: Сложные правила управления

In [11]:
print("\n=== Пример 5: Сложные правила ===")
# Создаем условия для разных режимов работы
comfort_temp = Fuzzy([0, 0, 0.2, 0.8, 1, 0.8, 0.2, 0, 0])    # Комфортная температура
high_humidity = Fuzzy([0, 0, 0, 0.2, 0.5, 0.8, 1, 1, 1])     # Высокая влажность
low_humidity = Fuzzy([1, 1, 1, 0.8, 0.5, 0.2, 0, 0, 0])      # Низкая влажность

# Сложное правило: (комфортная темп И НЕ высокая влаж) ИЛИ (НЕ комфортная темп И низкая влаж)
rule1 = comfort_temp & (~high_humidity)
rule2 = (~comfort_temp) & low_humidity
complex_rule = rule1 | rule2

print("Комфортная темп:", comfort_temp)
print("Высокая влаж:", high_humidity)
print("Низкая влаж:", low_humidity)
print("Сложное правило результат:", complex_rule)


=== Пример 5: Сложные правила ===
Комфортная темп: [0.  0.  0.2 0.8 1.  0.8 0.2 0.  0. ]
Высокая влаж: [0.  0.  0.  0.2 0.5 0.8 1.  1.  1. ]
Низкая влаж: [1.  1.  1.  0.8 0.5 0.2 0.  0.  0. ]
Сложное правило результат: [1.         1.         0.8        0.8        0.5        0.19999999
 0.         0.         0.        ]


In [12]:
#ЧАСТЬ 2: Работа с библиотекой fuzzywuzzy

In [14]:
#!pip install fuzzywuzzy python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

print("✅ Библиотеки fuzzywuzzy загружены!")

✅ Библиотеки fuzzywuzzy загружены!


In [15]:
#2.2. Примеры для системы управления сплит-системой

In [16]:
print("=== Пример 1: Сравнение команд ===")
user_command = "включи охлаждение"
possible_commands = ["включить охлаждение", "выключить охлаждение", 
                    "включи обогрев", "установи температуру", "включи вентиляцию"]

matches = process.extract(user_command, possible_commands, limit=3)
print(f"Команда: '{user_command}'")
print("Возможные совпадения:")
for match, score in matches:
    print(f"  • {match} - {score}%")

=== Пример 1: Сравнение команд ===
Команда: 'включи охлаждение'
Возможные совпадения:
  • включить охлаждение - 94%
  • выключить охлаждение - 92%
  • включи вентиляцию - 59%


In [17]:
#Пример 2: Распознавание режимов работы

In [18]:
print("\n=== Пример 2: Распознавание режимов ===")
modes = ["автоматический режим", "ночной режим", "экономичный режим", 
         "турбо режим", "тихий режим", "комфортный режим"]

user_input = "автомат"
best_match = process.extractOne(user_input, modes)
print(f"Пользователь ввел: '{user_input}'")
print(f"Лучшее совпадение: '{best_match[0]}' - {best_match[1]}%")

# Сравнение нескольких вариантов
user_variants = ["автомат", "ночь", "турбо", "комфорт"]
for variant in user_variants:
    match = process.extractOne(variant, modes)
    print(f"'{variant}' → '{match[0]}' ({match[1]}%)")


=== Пример 2: Распознавание режимов ===
Пользователь ввел: 'автомат'
Лучшее совпадение: 'автоматический режим' - 90%
'автомат' → 'автоматический режим' (90%)
'ночь' → 'ночной режим' (68%)
'турбо' → 'турбо режим' (90%)
'комфорт' → 'комфортный режим' (90%)


In [19]:
#Пример 3: Обработка температурных настроек

In [20]:
print("\n=== Пример 3: Температурные настройки ===")
temperature_commands = [
    "установи температуру 25 градусов",
    "поставь 23 градуса", 
    "сделай 20 градусов",
    "температура 18 градусов",
    "нагрей до 22 градусов"
]

user_request = "сделай 23 градуса"
ratios = []

for cmd in temperature_commands:
    ratio = fuzz.partial_ratio(user_request, cmd)
    token_ratio = fuzz.token_sort_ratio(user_request, cmd)
    w_ratio = fuzz.WRatio(user_request, cmd)
    
    ratios.append((cmd, ratio, token_ratio, w_ratio))

print(f"Запрос: '{user_request}'")
print("Сравнение различными методами:")
for cmd, ratio, token, w in ratios:
    print(f"  • {cmd:35} partial: {ratio:2}%, token: {token:2}%, WRatio: {w:2}%")


=== Пример 3: Температурные настройки ===
Запрос: 'сделай 23 градуса'
Сравнение различными методами:
  • установи температуру 25 градусов    partial: 59%, token: 49%, WRatio: 53%
  • поставь 23 градуса                  partial: 76%, token: 74%, WRatio: 74%
  • сделай 20 градусов                  partial: 88%, token: 86%, WRatio: 86%
  • температура 18 градусов             partial: 53%, token: 50%, WRatio: 50%
  • нагрей до 22 градусов               partial: 65%, token: 63%, WRatio: 60%


In [22]:
#Пример 4: Сравнение с игнорированием порядка слов

In [23]:
print("\n=== Пример 4: Сравнение с разным порядком слов ===")
commands_set1 = "включи кондиционер на охлаждение"
commands_set2 = [
    "охлаждение включи на кондиционер",
    "включи охлаждение кондиционера",
    "кондиционер включи режим охлаждение",
    "активируй охлаждение кондиционера"
]

for cmd in commands_set2:
    ratio = fuzz.ratio(commands_set1, cmd)
    token_sort = fuzz.token_sort_ratio(commands_set1, cmd)
    token_set = fuzz.token_set_ratio(commands_set1, cmd)
    
    print(f"'{commands_set1}' vs '{cmd}'")
    print(f"  Обычное: {ratio}%, Sort: {token_sort}%, Set: {token_set}%")


=== Пример 4: Сравнение с разным порядком слов ===
'включи кондиционер на охлаждение' vs 'охлаждение включи на кондиционер'
  Обычное: 56%, Sort: 100%, Set: 100%
'включи кондиционер на охлаждение' vs 'включи охлаждение кондиционера'
  Обычное: 61%, Sort: 97%, Set: 97%
'включи кондиционер на охлаждение' vs 'кондиционер включи режим охлаждение'
  Обычное: 69%, Sort: 87%, Set: 95%
'включи кондиционер на охлаждение' vs 'активируй охлаждение кондиционера'
  Обычное: 49%, Sort: 80%, Set: 80%


In [24]:
#Пример 5: Поиск в списке оборудования

In [25]:
print("\n=== Пример 5: Поиск оборудования ===")
equipment_list = [
    "сплит-система LG", "кондиционер Samsung", "вентилятор Bork",
    "сплит система Daikin", "кондиционер Ballu", "система Mitsubishi",
    "сплит-блок Gree", "кондиционер General", "система Panasonic"
]

search_queries = ["сплит система", "кондиционер", "система кондиционирования"]

for query in search_queries:
    print(f"\nПоиск: '{query}'")
    results = process.extract(query, equipment_list, limit=2)
    for result, score in results:
        print(f"  • {result} - {score}%")


=== Пример 5: Поиск оборудования ===

Поиск: 'сплит система'
  • сплит-система LG - 95%
  • сплит система Daikin - 90%

Поиск: 'кондиционер'
  • кондиционер Samsung - 90%
  • кондиционер Ballu - 90%

Поиск: 'система кондиционирования'
  • сплит-система LG - 86%
  • система Panasonic - 55%


In [26]:
#2.3. Интегрированный пример для системы управления

In [27]:
print("\n=== ИНТЕГРИРОВАННЫЙ ПРИМЕР ===")

class SmartACController:
    def __init__(self):
        self.commands_db = {
            "включи охлаждение": "cooling_on",
            "выключи охлаждение": "cooling_off", 
            "включи обогрев": "heating_on",
            "установи температуру": "set_temperature",
            "включи вентиляцию": "fan_on",
            "автоматический режим": "auto_mode",
            "ночной режим": "night_mode"
        }
    
    def process_command(self, user_input):
        # Нечеткое сравнение с базой команд
        best_match = process.extractOne(user_input, self.commands_db.keys())
        command, confidence = best_match
        
        if confidence > 70:
            action = self.commands_db[command]
            return f"Выполняю: {action} (уверенность: {confidence}%)"
        else:
            return f"Не понял команду. Лучшее совпадение: '{command}' ({confidence}%)"

# Тестирование контроллера
controller = SmartACController()
test_commands = [
    "включи охлождение",  # Опечатка
    "поставь ночной режим работы",
    "сделай автоматический",
    "выключи охлаждение пожалуйста"
]

for cmd in test_commands:
    result = controller.process_command(cmd)
    print(f"'{cmd}' → {result}")


=== ИНТЕГРИРОВАННЫЙ ПРИМЕР ===
'включи охлождение' → Выполняю: cooling_on (уверенность: 94%)
'поставь ночной режим работы' → Выполняю: night_mode (уверенность: 90%)
'сделай автоматический' → Выполняю: auto_mode (уверенность: 78%)
'выключи охлаждение пожалуйста' → Выполняю: cooling_off (уверенность: 90%)
